In [1]:
import os
import random
import numpy as np
from PIL import Image

In [2]:
folders = ['./trashnet-resized/paper', './trashnet-resized/cardboard',
           './trashnet-resized/glass', './trashnet-resized/metal',
           './trashnet-resized/plastic', './trashnet-resized/trash']

# Create a dictionary to map the class names to directories
class_dirs = {'paper': './mixup/paper', 'cardboard': './mixup/cardboard',
              'glass': './mixup/glass', 'metal': './mixup/metal',
              'plastic': './mixup/plastic', 'trash': './mixup/trash'}

# Runned once to add Mixed images to other images.
# class_dirs = {'paper': './trashnet-resized/paper', 'cardboard': './trashnet-resized/cardboard', 'glass': './trashnet-resized/glass', 'metal': './trashnet-resized/metal', 'plastic': './trashnet-resized/plastic', 'trash': './trashnet-resized/trash'}

In [3]:
def MixUp(images, labels):
    mixed_images = []
    mixed_labels = []

    label_dict = {label: [] for label in set(labels)}
    for i, label in enumerate(labels):
        label_dict[label].append(i)

    for i in range(int(len(images)/2)):
        # Choose a random label and two images with that label
        label = random.choice(list(label_dict.keys()))
        j, k = random.sample(label_dict[label], 2)

        # Generate a random mixing weight
        mix_weight = random.uniform(0.2, 0.8)

        # Mix the images using the given weight
        mixed_image = ((mix_weight * images[j]) + ((1 - mix_weight) * images[k])).astype(np.uint8)

        # Add the mixed image to the mixed_images list
        mixed_images.append(mixed_image)
        mixed_labels.append(labels[j])

    return mixed_images, mixed_labels

In [4]:
# Create a MixUp directory for each class
for class_dir in class_dirs.values():
    os.makedirs(class_dir, exist_ok=True)

# Loop over each folder and mix the images from the same class
for folder in folders:
    label = os.path.basename(folder)
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = np.array(Image.open(filepath))
        images.append(image)
    labels = [label] * len(images)
    mixed_images, mixed_labels = MixUp(images, labels)
    for i, mixed_image in enumerate(mixed_images):
        class_dir = class_dirs[mixed_labels[i]]
        filename = f"{i}.jpg"
        filepath = os.path.join(class_dir, filename)
        mixed_image = Image.fromarray(mixed_image)
        mixed_image.save(filepath)